In [9]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [10]:
df = pd.read_csv("Hate-2001-2019-0.csv")

In [11]:
#Fill empty values with None for Weapon Type
df['WeaponType'].fillna('None',inplace = True)

#Replace rest of NaN values with 'Unknown'
df_new = df.replace(np.nan, 'Unknown', regex=True)

#rearrange columns to be more easily readable
columns_titles = ['MostSeriousBias','MostSeriousBiasType','TotalNumberOfVictims','MostSeriousVictimType','TotalNumberOfSuspects','SuspectsRaceAsAGroup','MostSeriousUcr','MostSeriousUcrType','WeaponType','Offensive_Act','MostSeriousLocation','County','NCIC','ClosedYear','MonthOccurrence']
df_new=df_new.reindex(columns=columns_titles)

#One hot encode all necessary columns
df_new = pd.concat([df_new,pd.get_dummies(df_new['MostSeriousBiasType'], prefix='BiasType')],axis=1)
df_new.drop(['MostSeriousBiasType'],axis=1, inplace=True)
TotalNumberOfVictims = df_new['TotalNumberOfVictims']
df_new.drop(['TotalNumberOfVictims'],axis=1, inplace=True)
df_new = pd.concat([df_new,TotalNumberOfVictims],axis=1)
df_new = pd.concat([df_new,pd.get_dummies(df_new['MostSeriousVictimType'], prefix='VictimType')],axis=1)
df_new.drop(['MostSeriousVictimType'],axis=1, inplace=True)
TotalNumberOfSuspects = df_new['TotalNumberOfSuspects']
df_new.drop(['TotalNumberOfSuspects'],axis=1, inplace=True)
df_new = pd.concat([df_new,TotalNumberOfSuspects],axis=1)
df_new = pd.concat([df_new,pd.get_dummies(df_new['SuspectsRaceAsAGroup'], prefix='SuspectsRace')],axis=1)
df_new.drop(['SuspectsRaceAsAGroup'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['MostSeriousUcr'], prefix='Ucr')],axis=1)
df_new.drop(['MostSeriousUcr'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['MostSeriousUcrType'], prefix='UcrType')],axis=1)
df_new.drop(['MostSeriousUcrType'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['WeaponType'], prefix='Weapon')],axis=1)
df_new.drop(['WeaponType'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['Offensive_Act'], prefix='Offense')],axis=1)
df_new.drop(['Offensive_Act'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['MostSeriousLocation'], prefix='Location')],axis=1)
df_new.drop(['MostSeriousLocation'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['County'], prefix='County')],axis=1)
df_new.drop(['County'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['NCIC'], prefix='Agency')],axis=1)
df_new.drop(['NCIC'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['ClosedYear'], prefix='Year')],axis=1)
df_new.drop(['ClosedYear'],axis=1, inplace=True)
df_new = pd.concat([df_new,pd.get_dummies(df_new['MonthOccurrence'], prefix='Month')],axis=1)
df_new.drop(['MonthOccurrence'],axis=1, inplace=True)

In [12]:
df_new

,MostSeriousBias,BiasType_Disability,BiasType_Gender,BiasType_Gender Nonconforming,BiasType_Race/Ethnicity/Ancestry,BiasType_Religion,BiasType_Sexual Orientation,TotalNumberOfVictims,VictimType_Business,VictimType_Financial,VictimType_Government,VictimType_Other,VictimType_Person,VictimType_Religious,TotalNumberOfSuspects,SuspectsRace_American Indian or Alaska Native,SuspectsRace_Asian,SuspectsRace_Asian/Pacific Islander,SuspectsRace_Black or African American,SuspectsRace_East Indian,SuspectsRace_East Indian/Asian Indian,SuspectsRace_Group of Multiple Races,SuspectsRace_Hispanic,SuspectsRace_Native Hawaiian or Other Pacific Islander,SuspectsRace_Unknown,SuspectsRace_White,Ucr_Aggravated Assault,Ucr_Arson,Ucr_Burglary,Ucr_Destruction/Damage/Vandalism,Ucr_Intimidation,Ucr_Larceny-Theft,Ucr_Motor Vehicle Theft,Ucr_Murder,Ucr_Rape,Ucr_Robbery,Ucr_Simple Assault,UcrType_Property Crimes,UcrType_Violent Crimes,"Weapon_Arson, fire","Weapon_Blunt object (blugeon, club, etc.)","Weapon_Firearm (unknown whether handgun, rifle or shotgun","Weapon_Firearm (unknown whether handgun, rifle or shotgun)",Weapon_Handgun,Weapon_Knife or other cutting or stabbing instrument,Weapon_None,"Weapon_Other ( bottle, rocks, spitting)","Weapon_Other (bottle, rocks, spitting)","Weapon_Other gun (pellet, BB, stun gun, etc.)","Weapon_Personal weapons (hands, feet, teeth, etc.)",Weapon_Poison,Weapon_Rifle,Weapon_Ropes or garrote strangulation or hanging,Weapon_Shotgun,Weapon_Unknown,Weapon_Vehicle,Offense_Annoying telephone calls/fax,Offense_Bombing,Offense_Cross burning,Offense_Damage to vehicle,Offense_Daubing of swastika,Offense_Disturbing public assembly/meeting,Offense_Explosion,Offense_Graffiti,Offense_Hanging in Effigy,Offense_Other,Offense_Rock throwing,Offense_Threatening letters/flyers/email,Offense_Unknown,Offense_Verbal slurs,Location_ATM Separate from Bank,Location_Abandoned/Condemned Structure,Location_Air/Bus/Train Terminal,Location_Amusement Park,Location_Arena/Stadium/Fairgrounds/Coliseum,Location_Auto Dealership New/Used,Location_Bank/Savings and Loan,Location_Bar/Night Club,Location_Camp/Campground,Location_Church/Synagogue/Temple,Location_Commercial/Office Building,Location_Community Center,Location_Construction Site,Location_Convenience Store,Location_Daycare Facility,Location_Department/Discount Store,Location_Dock/Wharf/Freight/Modal Terminal,Location_Drug Store/Dr Office/Hospital,Location_Farm Facility,Location_Field/Woods/Park,Location_Gambling Facility/Casino/Race Track,Location_Government/Public Building,Location_Grocery/Supermarket,Location_Highway/Road/Alley/Street,Location_Hotel/Motel/etc.,Location_Industrial Site,Location_Jail/Prison,Location_Lake/Waterway/Beach,Location_Liquor Store,Location_Other/Unknown,Location_Park/Playground,Location_Parking Lot/Garage,Location_Rental Storage Facility,Location_Residence/Home/Driveway,Location_Rest Area,Location_Restaurant,Location_School-College/University,Location_School-Elementary/Secondary,Location_School/college,Location_Service/Gas Station,Location_Shelter-Mission/Homeless,Location_Shopping Mall,Location_Specialty Store (TV/Fur/etc.),Location_Tribal Lands,County_1,County_2,County_3,County_4,County_5,County_6,County_7,County_8,County_9,County_10,County_11,County_12,County_13,County_14,County_15,County_16,County_17,County_18,County_19,County_20,County_21,County_22,County_23,County_24,County_25,County_26,County_27,County_28,County_29,County_30,County_31,County_32,County_33,County_34,County_35,County_36,County_37,County_38,County_39,County_40,County_41,County_42,County_43,County_44,County_45,County_47,County_48,County_49,County_50,County_51,County_52,County_53,County_54,County_55,County_56,County_57,County_58,Agency_0101,Agency_0103,Agency_0104,Agency_0105,Agency_0106,Agency_0107,Agency_0109,Agency_0112,Agency_0113,Agency_0115,Agency_0130,Agency_0402,Agency_0404,Agency_0497,Agency_0701,Agency_0702,Agency_0704,Agency_0705,Agency_0708,Agency_0709,Agency_0710,Agency_0712,Agency_0714,Agency_072